In [1]:
import furuta_sympy_calc as furu_calc
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

# ==========================================================
# Parameter (flach, ohne Klassen)
# ==========================================================
g = 9.81

# Geometrie
L1 = 0.155
L2 = 0.240
lP = 0.1917

# Massen
mS = 0.0274
mP = 0.0  # 0.0510   # <- 0.0 wenn keine Zusatzmasse montiert ist
m2 = mS + mP

# Trägheitsmomente
J1_hat = 2.89e-2
J0_hat = J1_hat + m2 * L1**2

l2 = L2 / 2.0
J2_hat = (1 / 12) * mS * L2**2 + mS * l2**2 + mP * lP**2

# Reibungsparameter
mu_V1 = 50.91e-2
mu_V2 = 1.332e-5
haft_reibung = False
if haft_reibung:
    mu_H1 = 93.58e-3
    mu_H2 = 3.018e-4
else:
    mu_H1 = 0.0
    mu_H2 = 0.0

epsilon = 0.01
tau_max = 4.0

#sympy symbols
th1 = furu_calc.th1
th1d = furu_calc.th1d
th2 = furu_calc.th2
th2d = furu_calc.th2d
tau = furu_calc.tau

In [2]:
# non linear expressions
th1dd_expr, th2dd_expr = furu_calc.make_non_linear_furuta_accel_expr(
    g=g, L1=L1, l2=l2, J0_hat=J0_hat, J2_hat=J2_hat, m2=m2,
    mu_V1=mu_V1, mu_H1=mu_H1, mu_V2=mu_V2, mu_H2=mu_H2,
    epsilon=epsilon,
    simplify_expr=False,   # wenn's beim Start zu lange dauert -> False
)

a) Berechnen Sie die Systemmatrix A und den Eingangsvektor b mit den in Tab. 1
gegebenen Parametern.


In [3]:
f1 = th1d
f2 = th1dd_expr
f3 = th2d
f4 = th2dd_expr

f = sp.Matrix([f1, f2, f3, f4])
x = sp.Matrix([th1, th1d, th2, th2d])
u = sp.Matrix([tau])

A = f.jacobian(x)
b = f.jacobian(u)

A_lin = A.subs({th1:0, th1d:0, th2:0, th2d:0, tau:0})
b_lin = b.subs({th1:0, th1d:0, th2:0, th2d:0, tau:0})

A_lin

Matrix([
[0,                 1,                 0,                    0],
[0, -17.5161709980497,  1.07509937893889, 0.000443968358900185],
[0,                 0,                 0,                    1],
[0,  16.9687906543607, -62.3540025233471,   -0.025749437413378]])

In [4]:
b_lin

Matrix([
[                0],
[  34.406150064918],
[                0],
[-33.3309578753893]])

b) Überprüfen Sie die Steuerbarkeit des linearisierten Modells.

In [34]:
# SymPy -> NumPy
A_np = np.array(A_lin.tolist(), dtype=float)
b_np = np.array(b_lin.tolist(), dtype=float)

n = A_np.shape[0]

# Steuerbarkeitsmatrix C = [B, AB, A^2B, ...]
S_blocks = [np.linalg.matrix_power(A_np, k) @ b_np for k in range(n)] #liste von spalten vektoren
S_np = np.hstack(C_blocks) #spaletenvektoren werden zu matrix zusammen gefügt

rankS = np.linalg.matrix_rank(C_np)  # nutzt intern Toleranz (SVD-basiert)

print("S shape:", S_np.shape)
print("rank(S):", rankS)
print("voll steuerbar?", rankS == n)

S shape: (4, 4)
rank(S): 4
voll steuerbar? True


c) Berechnen Sie die Eigenwerte der Systemmatrix.

In [35]:
np.linalg.eig(A_np)

EigResult(eigenvalues=array([  0.        +0.j        , -17.46681993+0.j        ,
        -0.03755025+7.84119401j,  -0.03755025-7.84119401j]), eigenvectors=array([[ 1.00000000e+00+0.j        , -4.44670392e-02+0.j        ,
        -8.23695775e-04+0.00037581j, -8.23695775e-04-0.00037581j],
       [ 0.00000000e+00+0.j        ,  7.76697767e-01+0.j        ,
        -2.91590107e-03-0.00647287j, -2.91590107e-03+0.00647287j],
       [ 0.00000000e+00+0.j        ,  3.59123592e-02+0.j        ,
        -6.05792834e-04-0.12650086j, -6.05792834e-04+0.12650086j],
       [ 0.00000000e+00+0.j        , -6.27274712e-01+0.j        ,
         9.91940495e-01+0.j        ,  9.91940495e-01-0.j        ]]))

d) Überführen Sie das linearisierte System in Regelungsnormalform (RNF) und führen 
Sie einen flachen Ausgang ein.


In [36]:
S_inv = np.linalg.inv(S_np)
q1T = S_inv[-1, :].reshape(1, -1) #reshape to explicitly make it a row vector

In [44]:
Q_blocks = [q1T @ np.linalg.matrix_power(A_np, k)for k in range(n)] #liste von zeilen vektoren
Q = np.vstack(Q_blocks)
Q_inv = np.linalg.inv(Q)
tol = 1e-13   # wählbar, z.B. 1e-12, 1e-9, je nach Skalierung

A_np[np.abs(A_np) < tol] = 0.0 

A_RNF = Q @ A_np @ Q_inv
A_RNF[np.abs(A_RNF) < tol] = 0.0
A_RNF

array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 0.00000000e+00, -1.07396023e+03, -6.27975005e+01,
        -1.75419204e+01]])

In [45]:
b_RNF = Q @ b_np
b_RNF[np.abs(b_RNF) < tol] = 0.0
b_RNF


array([[0.],
       [0.],
       [0.],
       [1.]])

In [46]:
q1T

array([[ 4.74039898e-04, -1.95757452e-07,  4.89326391e-04,
        -2.02072209e-07]])